## Cat or dog?

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2
import pandas as pd
from keras.models import Model
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.preprocessing.image import ImageDataGenerator
from google.colab import drive
from keras.applications.vgg16 import VGG16

Connecting to gdrive to acquire data

In [ ]:
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


Specifying location of needed files and defining constants

In [ ]:
data_dir = 'gdrive/MyDrive/sieci/train'
test_dir = 'gdrive/MyDrive/sieci/test'
img_size = 128 # do sieci będziemy podawać obrazy o rozmiarze 128x128 (obrazy w danych wejściowych mają różne rozmiary)
batch_size = 32 # liczba zdjęć na jeden krok uczenia (tu nie da się podać całego zbioru na raz ze względu na RAM)

Dividing pictures into learning and validation *data*

In [ ]:
train = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.3,
    subset="training",
    seed=0,
    label_mode='binary',
    image_size=(img_size, img_size),
    batch_size=batch_size,
    color_mode='rgb'
)
val = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.3,
    subset="validation",
    seed=0,
    label_mode='binary',
    image_size=(img_size, img_size),
    batch_size=batch_size,
    color_mode='rgb'
)

test = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir,
    image_size=(img_size, img_size),
    batch_size=batch_size,
    color_mode='rgb',
    labels=None,
    shuffle = False
)

Found 18737 files belonging to 2 classes.
Using 13116 files for training.
Found 18737 files belonging to 2 classes.
Using 5621 files for validation.
Found 4685 files belonging to 1 classes.


Data augmentation

In [ ]:
train_datagen = ImageDataGenerator(rescale=1.0/255.0, shear_range=0.2, 
    vertical_flip=True, horizontal_flip=True, validation_split=0.3)

test_datagen = ImageDataGenerator(rescale=1.0/255.0)

train_generator = train_datagen.flow_from_directory(
    data_dir,
    subset="training",
    seed=0,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='binary',
    color_mode='rgb')

validation_generator = train_datagen.flow_from_directory(
    data_dir,
    subset="validation",
    seed=0,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='binary',
    color_mode='rgb')

Found 13117 images belonging to 2 classes.
Found 5620 images belonging to 2 classes.


Making sure that best model will be saved

In [ ]:
checkpoint_filepath = 'gdrive/MyDrive/sieci/checkpoint1.hdf5'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

Creating a model based on VGG16

In [ ]:
#model = tf.keras.models.Sequential()
#model.add(tf.keras.layers.experimental.preprocessing.Rescaling(1/255, input_shape=(img_size, img_size, 3)))
#model.add(Conv2D(32, (3, 3), activation='LeakyReLU', kernel_initializer='he_uniform', padding='same'))
#model.add(MaxPooling2D((2, 2), strides = (2, 2)))
#model.add(Dropout(0.3))
#model.add(Conv2D(64, (3, 3), activation='LeakyReLU', kernel_initializer='he_uniform', padding='same'))
#model.add(MaxPooling2D((2, 2), strides = (2, 2)))
#model.add(Dropout(0.3))
#model.add(Conv2D(128, (3, 3), activation='LeakyReLU', kernel_initializer='he_uniform', padding='valid'))
#model.add(MaxPooling2D((2, 2), strides = (2, 2)))
#model.add(Dropout(0.3))
#model.add(tf.keras.layers.Flatten())
#model.add(Dense(128, activation='LeakyReLU'))
#model.add(Dropout(0.3))
#model.add(Dense(1, activation='sigmoid'))
#model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# TODO sieć konwolucyjna

model = VGG16(include_top=False, input_shape=(img_size, img_size, 3))
for layer in model.layers:
  layer.trainable = False
flat1 = Flatten()(model.layers[-1].output)
class1 = Dense(128, activation='relu', kernel_initializer='he_uniform')(flat1)
output = Dense(1, activation='sigmoid')(class1)
model = Model(inputs=model.inputs, outputs=output)
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

Model summary

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 128, 128, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 128, 128, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 64, 64, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 64, 64, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 64, 64, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 32, 32, 128)       0     

Teaching model on additional and original data separetly

In [ ]:
history = model.fit_generator(train_generator, steps_per_epoch=len(train_generator), epochs=5,
                              validation_data=validation_generator, validation_steps=len(validation_generator),
                              callbacks=[model_checkpoint_callback])
history = model.fit(train, epochs=10, validation_data=val, callbacks=[model_checkpoint_callback])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  """


Epoch 1/5
410/410 [==============================] - 3258s 8s/step - loss: 0.5150 - accuracy: 0.7686 - val_loss: 0.3461 - val_accuracy: 0.8443
Epoch 2/5
 61/410 [===>..........................] - ETA: 31:45 - loss: 0.3869 - accuracy: 0.8204

Loading best model

In [ ]:
model = load_model(checkpoint_filepath)

Making the VGG16 basis trainable for 5 last epochs, for best resoults 

In [ ]:
for layer in model.layers:
  layer.trainable = True

In [ ]:
history = model.fit(train, epochs=5, validation_data=val, callbacks=[model_checkpoint_callback])

Loading best model and making predictions

In [ ]:
model = load_model(checkpoint_filepath)

In [ ]:
model.evaluate(train)

In [ ]:
model.evaluate(val)

In [ ]:
test.file_paths[0].split('/')[-1]

In [ ]:
preds = model.predict(test) 
preds = preds >= 0.5

preds = list(preds)
df_result = pd.DataFrame(list(zip([f.split('/')[-1] for f in test.file_paths], preds)),
                         columns=['file', 'label'])

df_result['label'] = df_result['label'].astype(int)
df_result.head()

Saving resoults

In [ ]:
df_result.to_csv('gdrive/MyDrive/sieci/submission.csv', sep=',', index=False)

Useful graphs

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.legend(['dokładność na zbiorze uczącym', 'dokładność na zbiorze walidacyjnym']);

In [ ]:
# wyświetlanie przebiegu uczenia (loss - funkcja straty)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['funkcja straty na zbiorze uczącym', 'funkcja straty na zbiorze walidacyjnym']);

In [ ]:
# utworzony model można sobie zapisać
# https://www.tensorflow.org/guide/keras/save_and_serialize
# proszę zmienić ścieżkę
model.save('gdrive/MyDrive/sieci/m1.h5')

In [ ]:
# oraz wczytać
model = tf.keras.models.load_model('gdrive/MyDrive/sieci/m1.h5')

Prediction for specific file

In [ ]:
# można posprawdzać różne zdjęcia (również dowolne własne, niekoniecznie ze zbioru), dla jakich klasyfikator sobie nie radzi?
# ścieżka musi się zgadzać z tym gdzie mamy dane
path = 'gdrive/MyDrive/sieci/train/Dog/100.jpg'

In [ ]:
image = tf.keras.preprocessing.image.load_img(path, target_size=(img_size, img_size))
input_arr = tf.keras.preprocessing.image.img_to_array(image)
input_arr_batch = np.array([input_arr])
predictions = model.predict(input_arr_batch)

In [ ]:
# 0 - cat
# 1 - dog
predictions

In [ ]:
img = cv2.imread(path)
plt.imshow(img)
plt.show()